# Deploying a web service to Azure Kubernetes Service (AKS)

In this notebook, we show the following steps for deploying a web service using AML:

- Provision an AKS cluster (one time action)
- Deploy the service
- Test the web service

In [ ]:
import json
import subprocess

import matplotlib.pyplot as plt
import requests
from azureml.core.compute import AksCompute, ComputeTarget
from azureml.core.webservice import Webservice, AksWebservice
from dotenv import set_key, get_key, find_dotenv
from testing_utilities import read_image_from
from testing_utilities import to_img, get_auth
from MetricsUtils.hpStatisticsCollection import statisticsCollector, CollectionEntry


In [ ]:
env_path = find_dotenv(raise_error_if_not_found=True)

In [ ]:
image_name = get_key(env_path, "image_name")

In [ ]:
aks_service_name = "YOUR_AKS_SERVICE_NAME"
aks_name = "YOUR_AKS_NAME"
aks_location = "YOUR_AKS_LOCATION"

In [ ]:
set_key(env_path, "aks_service_name", aks_service_name)
set_key(env_path, "aks_name", aks_name)
set_key(env_path, "aks_location", aks_location)

<a id='get_workspace'></a>
## Get workspace
Load existing workspace from the config file info.

In [ ]:
from azureml.core.workspace import Workspace

ws = Workspace.from_config(auth=get_auth())
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep="\n")

Restore the statistics data.

In [ ]:
storageConnString = get_key(env_path, "storage_conn_string")
statisticsCollector.hydrateFromStorage(storageConnString)

<a id='provision_cluster'></a>
## Provision the AKS Cluster¶
This is a one time setup. You can reuse this cluster for multiple deployments after it has been created. If you delete the cluster or the resource group that contains it, then you would have to recreate it. Let's first check if there are enough cores in the subscription for the cluster.

In [ ]:
vm_dict = {"NC": {"size": "Standard_NC6", "cores": 6}}

In [ ]:
vm_family = "NC"
node_count = 3  # We need to have a minimum of 3 nodes
requested_cores = node_count * vm_dict[vm_family]["cores"]

In [ ]:
results = subprocess.run(
    [
        "az",
        "vm",
        "list-usage",
        "--location",
        get_key(env_path, "aks_location"),
        "--query",
        "[?contains(localName, '%s')].{max:limit, current:currentValue}" % (vm_family),
    ],
    stdout=subprocess.PIPE,
)
print(results.stdout.decode("utf-8"))
quota = json.loads("".join(results.stdout.decode("utf-8")))
diff = int(quota[0]["max"]) - int(quota[0]["current"])

In [ ]:
if diff <= requested_cores:
    print(
        "Not enough cores of NC6 in region, asking for {} but have {}".format(
            requested_cores, diff
        )
    )
    raise Exception("Core Limit", "Note enough cores to satisfy request")
print("There are enough cores, you may continue...")

In [ ]:
%%time
from azureml.core.compute_target import ComputeTargetException

# Provision AKS cluster with GPU machine
prov_config = AksCompute.provisioning_configuration(vm_size="Standard_NC6")
statisticsCollector.startTask(CollectionEntry.AML_COMPUTE_CREATION)

def create_and_wait(retry_count):
    # Create the cluster, retry if failed.
    try: 
        aks_target = ComputeTarget.create(
            workspace=ws, name=aks_name + str(retry_count), provisioning_configuration=prov_config
        )
        
        aks_target.wait_for_completion(show_output=True)
        print(aks_target.provisioning_state)
        print(aks_target.provisioning_errors)
        return {'succeeded': True, 'aks_target': aks_target}
    except ComputeTargetException as ex:
        print(ex)
        return {'succeeded': False, 'aks_target': None}

def retry_func(func, retry_count=0):
    succeeded = False
    result = {}
    while not succeeded:        
        if retry_count < 1: # Retry 1 time
            result = func(retry_count)
            succeeded = result['succeeded']
        else:
            raise Exception("Tried to create AKS 3 times and failed!")
        retry_count += 1
        print("Retry: " + str(retry_count))
    return result['aks_target']
        
aks_target = retry_func(create_and_wait)

statisticsCollector.endTask(CollectionEntry.AML_COMPUTE_CREATION)
print(statisticsCollector.getEntry(CollectionEntry.AML_COMPUTE_CREATION))

In [ ]:
# Attach an existing AKS cluster

# attach_config = AksCompute.attach_configuration(resource_group=ws.resource_group,
#                                                cluster_name='deployaks')
# aks_target = ComputeTarget.attach(ws, aks_name, attach_config)
# aks_target.wait_for_completion(True)

In [ ]:
# Execute following commands if you want to delete an AKS cluster
# aks_target = AksCompute(name=aks_name,workspace=ws)
# aks_target.delete()

<a id='deploy_ws'></a>
## Deploy web service to AKS¶

In [ ]:
# Deploy web service to AKS
# Set the web service configuration (using customized configuration)
aks_config = AksWebservice.deploy_configuration(autoscale_enabled=False, num_replicas=node_count)

In [ ]:
# get the image built in previous notebook
image = ws.images[image_name]

In [ ]:
aks_service_name

In [ ]:
aks_service = Webservice.deploy_from_image(
    workspace=ws,
    name=aks_service_name,
    image=image,
    deployment_config=aks_config,
    deployment_target=aks_target,
)

In [ ]:
%%time
aks_service.wait_for_deployment(show_output=True)
print(aks_service.state)

In [ ]:
### debug
# aks_service.error
# aks_service.get_logs()

# Excute following commands if you want to delete a web service
# s =  Webservice(ws, aks_service_name)
# s.delete()

Write the URI and key to the statistics tracker.

In [ ]:
scoring_url = aks_service.scoring_uri
api_key = aks_service.get_keys()[0]

In [ ]:
statisticsCollector.addEntry(CollectionEntry.AKS_REALTIME_ENDPOINT, scoring_url)
statisticsCollector.addEntry(CollectionEntry.AKS_REALTIME_KEY, api_key)

<a id='test_ws'></a>
## Test Web Service¶
We test the web sevice by passing data.

In [ ]:
IMAGEURL = "https://bostondata.blob.core.windows.net/aksdeploymenttutorialaml/220px-Lynx_lynx_poing.jpg"
plt.imshow(to_img(IMAGEURL))

In [ ]:
service_keys = aks_service.get_keys()
headers = {}
headers["Authorization"] = "Bearer " + service_keys[0]

In [ ]:
resp = requests.post(
    aks_service.scoring_uri,
    headers=headers,
    files={"image": read_image_from(IMAGEURL).read()},
)

In [ ]:
print(resp.json())

Save the statistics collected so far.

In [ ]:
statisticsCollector.uploadContent(storageConnString)

Having deplied web service succesfully, we can now move on to [Test Web app](05_TestWebApp.ipynb).